In [415]:
#!pip install psycopg2-binary

In [416]:
#!pip install category_encoders

In [417]:
#!pip install catboost

In [418]:
#!git clone https://github.com/labs15-baseball-pitch-predictor/RC-v1.1--Data-Cleaning-and-Feature-Engineering.git

In [419]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
import psycopg2
#import category_encoders as ce

from sklearn.preprocessing import RobustScaler, StandardScaler
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [420]:
pwd

'/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/notebooks'

## Load in the pre-pitcher preprocessed_df



In [421]:
#load in pickled preprocessed df
path1 = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl'
path2 = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl'
df1 = pd.read_pickle(path1, compression='zip')
df2 = pd.read_pickle(path2, compression='zip')

df = pd.concat([df1, df2])

In [422]:
#clear some memory:
del df1
del df2
gc.collect()

146

In [423]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

def make_pitcher_list(pre_processed_df):
    #make list of all the pitcher ids in the preprocessed_df
    pitcher_list = df['pitcher'].unique().tolist()
    
    #on rare occassions in extra-inning games, a position player is called in to pitch: 
    #to avoid categorizing him as a pitcher we'll only count pitchers with over 100 pitches:
    for pitcher in pitcher_list:
        if len(df[df['pitcher'] == pitcher]) < 100:
            pitcher_list.remove(pitcher)
    return pitcher_list

pitcher_list = make_pitcher_list(df)

## Choose the pitcher to model

In [424]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [425]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
# pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
# pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
# pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.5210,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.2140,-8.9025,30.410400,-12.247200,3.6311,1.5714,NaN,NaN,NaN,92.084000,2213.0,6.479,529438.0,54.021900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball
10019,FT,2018-03-31,15181,91.199997,-2.2699,6.3632,448801.0,502043.0,8.0,1.0,2018.0,-1.4111,1.0147,-0.2476,1.8754,0.0,0.0,0.0,0.0,1.0,130.669998,195.059998,488771.0,8.2048,-132.487595,-7.6040,-18.5795,29.222300,-18.640699,3.5538,1.5042,23.0,14.3,-21.0,91.592003,1961.0,6.692,529438.0,53.808201,0.15,0.138,0.0,1.0,0.0,0.0,2.0,5.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,X,ground_ball,Bot,180331_231716,Infield shift,1.0,NaN,False,False,False,False,0,1,1,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,field_out,hit_into_play,"Chris Davis grounds out softly, pitcher Kyle G...",2-Seam Fastball,Standard,10,behind,fastball
10020,FF,2018-03-31,15167,92.900002,-1.8539,6.4785,592518.0,502043.0,9.0,NaN,2018.0,-0.6207,1.6913,0.6782,1.8998,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,7.9316,-134.882996,-9.7386,-9.4737,30.538200,-9.245200,3.8157,1.8805,NaN,NaN,NaN,93.452003,2244.0,6.831,529438.0,53.669300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,S,NaN,Bot,180331_231759,Standard,1.0,NaN,False,False,False,False,0,0,1,0,60.031658,30.121817,43.618515,28.175896,0.439553,0.199495,0.176768,30.668777,38.163265,39.130436,28.421053,0.371922,0.163551,0.186916,9.259992,40.397350,40.740742,10.843373,0.425091,0.205479,0.315068,0.039573,0,0,Kyle Gibson,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball
10021,SL,2018-03-31,15153,85.199997,-2.1695,6.3682,592518.0,502043.0,14.0,NaN,2018.0,0.6245,0.2967,2.3338,1.7924,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,9.2954,-123.751099,-4.9790,4.4722,25.264400,-28.358299,3.6814,1.7561,NaN,NaN,NaN,84.822998,2497.0,6.199,529438.0,54.301300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,B,NaN,Bot,180331_231814,Standard,1.0,NaN,Fa

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [426]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id

df_17_path = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl'
df17 = pd.read_pickle(df_17_path, compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 1.88 s, sys: 316 ms, total: 2.2 s
Wall time: 2.2 s


In [427]:
def get_left_right_pitch_tendencies(pitcher_df):
    #split the df into left hand and right handed batters
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    overall_left = left['pitch_cat'].value_counts(normalize=True).to_dict()
    overall_right = right['pitch_cat'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    by_count_left = {}
    by_count_right = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        left_subset = left[left['count_cat'] == cat]
        right_subset = right[right['count_cat'] == cat]
        
        by_count_left[cat] = left_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        by_count_right[cat] = right_subset['pitch_cat'].value_counts(normalize=True).to_dict()
        
    return overall_left, overall_right, by_count_left, by_count_right

def make_tendency_features(pitcher_df, overall_left, overall_right, by_count_left, by_count_right):
    #helper functions to vectorize w/ df.apply():
    def get_overall_left_perc(x):
        return overall_left[x] * 100
    def get_overall_right_perc(x):
        return overall_right[x] * 100
    def get_by_count_left_perc(x):
        try:
            return by_count_left[x][pitch_type] * 100
        except KeyError:
            return 0
    def get_by_count_right_perc(x):
        try:
            return by_count_right[x][pitch_type] * 100
        except KeyError:
            return 0
    
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    pitch_types_left = overall_left.keys()
    pitch_types_right = overall_right.keys()
    
    #Left
    for pitch_type in pitch_types_left:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        left[overall_feature] = pitch_type
        left[overall_feature] = left[overall_feature].apply(get_overall_left_perc)
        left[count_cat_feature] = left['count_cat'].apply(get_by_count_left_perc)
    #Right
    for pitch_type in pitch_types_right:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        right[overall_feature] = pitch_type
        right[overall_feature] = right[overall_feature].apply(get_overall_right_perc)
        right[count_cat_feature] = right['count_cat'].apply(get_by_count_right_perc)
        
    
    return pd.concat([left,right], sort=False).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])

def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])].copy()
        
        #get the pitch tendencies from prior:
        overall_left, overall_right, by_count_left, by_count_right = get_left_right_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, overall_left, overall_right, by_count_left, by_count_right)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df

In [428]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

In [429]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 1.15 s, sys: 4 ms, total: 1.15 s
Wall time: 1.15 s


In [430]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.5210,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.2140,-8.9025,30.410400,-12.247200,3.6311,1.5714,NaN,NaN,NaN,92.084000,2213.0,6.479,529438.0,54.021900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.780908,64.197531,20.422535,19.341564,19.796557,16.460905
10019,FT,2018-03-31,15181,91.199997,-2.2699,6.3632,448801.0,502043.0,8.0,1.0,2018.0,-1.4111,1.0147,-0.2476,1.8754,0.0,0.0,0.0,0.0,1.0,130.669998,195.059998,488771.0,8.2048,-132.487595,-7.6040,-18.5795,29.222300,-18.640699,3.5538,1.5042,23.0,14.3,-21.0,91.592003,1961.0,6.692,529438.0,53.808201,0.15,0.138,0.0,1.0,0.0,0.0,2.0,5.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,X,ground_ball,Bot,180331_231716,Infield shift,1.0,NaN,False,False,False,False,0,1,1,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,field_out,hit_into_play,"Chris Davis grounds out softly, pitcher Kyle G...",2-Seam Fastball,Standard,10,behind,fastball,59.780908,73.913043,20.422535,6.719368,19.796557,19.367589
10020,FF,2018-03-31,15167,92.900002,-1.8539,6.4785,592518.0,502043.0,9.0,NaN,2018.0,-0.6207,1.6913,0.6782,1.8998,0.0,0.0,0.0,1.0,1.0,NaN,NaN,488771.0,7.9316,-134.882996,-9.7386,-9.4737,30.538200,-9.245200,3.8157,1.8805,NaN,NaN,NaN,93.452003,2244.0,6.831,529438.0,53.669300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,BAL,MIN,S,NaN,Bot,180331_231759,Standard,1.0,NaN,False,False,False,False,0,0,1,0,60.031658,30.121817,43.618515,28.175896,0.439553,0.199495,0.176768,30.668777,38.163265,39.130436,28.421053,0.371922,0.163551,0.186916,9.259992,40.397350,40.740742,10.843373,0.425091,0.205479,0.315068,0.039573,0,0,Kyle Gibson,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,56.006006,62.785863,34.459459,30.977131,9.534535,6.237006
10021,SL,2018-03-31,15153,85.199997,-2.1695,6.3682,592518.0,502043.0,14.0,NaN,201

In [431]:
pitcher_df.description.value_counts()

ball                       2008
foul                        869
called_strike               862
hit_into_play               637
swinging_strike             607
hit_into_play_no_out        245
blocked_ball                228
hit_into_play_score         130
swinging_strike_blocked      95
foul_tip                     34
foul_bunt                    20
hit_by_pitch                 11
missed_bunt                   3
Name: description, dtype: int64

## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [432]:
pitcher_df.type.head()

10018    B
10019    X
10020    S
10021    B
10022    B
Name: type, dtype: category
Categories (3, object): [B, S, X]

In [433]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 10 (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 10
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    #make column for if the bater is a pitcher, also use battting order slot number as 11
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: 1 if x in pitcher_list else 0)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == 0, other=11)
    return game_df

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_cat'].value_counts(normalize=True).to_dict()

    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_cat'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    all_games = []
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    for game in games:
        
        #take the first game and make the pitch count feature
        game_df = df[df['game_pk'] == game].copy()
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1)
    
        #make the L1 pitch features:
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_description'] = game_df['description'].shift(periods=1)
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_batter_swung'] = game_df['batter_swung'].shift(periods=1)
        game_df['L1_chased'] = game_df['chased'].shift(periods=1)        
        
        #add L2 and L3 features:
        
        game_df['L2_pitch_type'] = game_df['pitch_type'].shift(periods=2)
        game_df['L2_pitch_result'] = game_df['type'].shift(periods=2)
        game_df['L2_description'] = game_df['description'].shift(periods=2)
        game_df['L2_pitch_zone'] = game_df['zone'].shift(periods=2)
        game_df['L2_chased'] = game_df['chased'].shift(periods=2)
        
        game_df['L3_pitch_type'] = game_df['pitch_type'].shift(periods=3)
        game_df['L3_description'] = game_df['description'].shift(periods=3)
        game_df['L3_pitch_zone'] = game_df['zone'].shift(periods=3)
        game_df['L3_chased'] = game_df['chased'].shift(periods=3)
        
        str_shifted_cols = ['L1_pitch_type', 'L1_pitch_result','L1_description', 'L2_pitch_type', 
                            'L2_pitch_result','L2_description', 'L3_pitch_type','L3_description']
        
        num_shifted_cols = ['L1_batter_swung', 'L1_chased', 'L1_pitch_zone', 'L2_chased',
                            'L2_pitch_zone', 'L3_chased', 'L3_pitch_zone']
        
        #fill Nan for shifted cols w/ string N/A
        game_df[str_shifted_cols] = game_df[str_shifted_cols].replace({np.nan:'N/A'})
        game_df[num_shifted_cols] = game_df[num_shifted_cols].replace({np.nan:-1})
        
    
        #overall strike % (to fill in for first 5 pitches L5_strike_perc)
        overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100

        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #for the first 5 rows, use overall pitcher tendencies    
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    feature = 'L15_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                #strike %
                game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
                game_df.at[index, 'L15_strike_perc'] = overall_strike_perc

            else:
                current_pitch = game_df.at[index, 'pitch_count']
                #make a subset of the prev 5 pitches
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                #grab the value count percentages for the last 5 pitches
                subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0

                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc

                #iterate over all possible pitch types this pitcher throws:
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    #if he has thrown that pitch type in last 5
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    #except for when he hasnt thrown that type in last 5
                    except:
                        game_df.at[index, feature] = 0

                if row['pitch_count'] < 16:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 15 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 15
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                else:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] > current_pitch - 16) & (game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 5 pitches
                    subset_percentages = subset['pitch_cat'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 5
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                            
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
    
        all_games.append(game_df)
    
    new_df = pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

def make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df):
    
    all_games = []
    #iterate over each game
    for game in pitcher_df['game_pk'].unique():
        #make subset df for that game
        game_df = pitcher_df[pitcher_df['game_pk'] == game].copy()
        #initialize columns to False:
        game_df['prev_ab_run_scored'] = 0
        game_df['prev_ab_homerun'] = 0
        game_df['prev_ab_walk'] = 0
        game_df['prev_ab_basehit'] = 0
        game_df['prev_ab_strikeout'] = 0
        
        #this gets the 
        at_bats = game_df['at_bat_number'].sort_values().unique()
        
        #initialize empty dicts
        run_scored = []
        homeruns = []
        walks = []
        basehits = []
        strikeouts = []
        
        walks = ['walk', 'hit_by_pitch']
        basehits = ['single', 'double', 'triple', 'home_run']
        
        #starting w/ 2nd AB, iterate thru to the end of the at_bats:
        for ab in at_bats[2:]:
            #get the index for the last pitch of the prev AB
            prev_ab_last_pitch_index = game_df[game_df['at_bat_number'] == ab-1]['pitch_number'].index.max()
            #check if the last pitch resulted in a walk or hit by pitch:
            if game_df.loc[prev_ab_last_pitch_index]['events'] in walks:
                #if so, add an entry
                walks.append(ab)
            #check if last pitch gave up a basehit:
            elif game_df.loc[prev_ab_last_pitch_index]['events'] in basehits:
                basehits.append(ab)
            elif game_df.loc[prev_ab_last_pitch_index]['events'] == 'strikeout':
                strikeouts.append(ab)
            
            #to check if prev AB resulted in a run scoring: compare score before and after the AB
            prev_score = game_df[game_df['at_bat_number'] == ab-1]['bat_score'].values[0]
            current_score = game_df[game_df['at_bat_number'] == ab]['bat_score'].values[0]
            
            if current_score > prev_score:
                run_scored.append(ab)
                 
                #check if last AB gave up a homerun:
                if game_df.loc[prev_ab_last_pitch_index]['events'] == 'home_run':
                    homeruns.append(ab)
                    
        #iterate over each at_bat, and add the features to the df where appropriate
        for ab in at_bats:
            idx = game_df[game_df['at_bat_number'] == ab].index
            if ab in walks:
                game_df.at[idx, 'prev_ab_walk'] = 1
            elif ab in basehits:
                game_df.at[idx, 'prev_ab_basehit'] = 1
            elif ab in strikeouts:
                game_df.at[idx, 'prev_ab_strikeout'] = 1
            if ab in run_scored:
                game_df.at[idx, 'prev_ab_run_scored'] = 1
                if ab in homeruns:
                    game_df.at[idx, 'prev_ab_homerun'] = 1
        all_games.append(game_df)
        
    return pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'pitch_count'])

In [434]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list)

#pitches in df before: 5749
# pitches in df after: 5749
CPU times: user 1min 5s, sys: 7.76 ms, total: 1min 5s
Wall time: 1min 5s


In [435]:
%%time
pitcher_df = make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df)

CPU times: user 17.6 s, sys: 7.94 ms, total: 17.6 s
Wall time: 17.6 s


In [436]:
pitcher_df[['description', 'type', 'events','des', 'batter_swung', 'chased']].head(10)

,description,type,events,des,batter_swung,chased
10018,ball,B,NaN,NaN,0,0
10019,hit_into_play,X,field_out,"Chris Davis grounds out softly, pitcher Kyle G...",1,0
10020,called_strike,S,NaN,NaN,0,0
10021,ball,B,NaN,NaN,0,0
10022,ball,B,NaN,NaN,0,0
10023,ball,B,NaN,NaN,0,0
10024,foul,S,NaN,NaN,1,0
10025,ball,B,walk,Manny Machado walks.,0,0
10026,called_strike,S,NaN,NaN,0,0
10027,foul,S,NaN,NaN,1,1


In [437]:
cols = ['pitch_count', 'pitch_type', 'type', 'batting_order_slot']

for col in pitcher_df.columns:
    if col[:1] == 'L':
        cols.append(col)
    if col[-4:] == 'perc':
        if col[:2]=='L5':
            cols.append(col)
        if col[:3]=='L15':
            cols.append(col)
        
pitcher_df[cols].head(20)

,pitch_count,pitch_type,type,batting_order_slot,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L5_fastball_perc,L15_fastball_perc,L15_fastball_perc,L5_breaking_perc,L5_breaking_perc,L15_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L5_offspeed_perc,L15_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L5_strike_perc,L15_strike_perc,L15_strike_perc
10018,1,FF,B,1,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,54.652983,54.652983,32.440424,32.440424,32.440424,32.440424,12.906592,12.906592,12.906592,12.906592,43.31188,43.31188,43.311880,43.311880
10019,2,FT,X,1,FF,B,ball,14.0,0.0,0.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,54.652983,54.652983,32.440424,32.440424,32.440424,32.440424,12.906592,12.906592,12.906592,12.906592,43.31188,43.31188,43.311880,43.311880
10020,3,FF,S,2,FT,X,hit_into_play,8.0,1.0,0.0,FF,B,ball,14.0,0.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,54.652983,54.652983,32.440424,32.440424,32.440424,32.440424,12.906592,12.906592,12.906592,12.906592,43.31188,43.31188,43.311880,43.311880
10021,4,SL,B,2,FF,S,called_strike,9.0,0.0,0.0,FT,X,hit_into_play,8.0,0.0,FF,ball,14.0,0.0,54.652983,54.652983,54.652983,54.652983,32.440424,32.440424,32.440424,32.440424,12.906592,12.906592,12.906592,12.906592,43.31188,43.31188,43.311880,43.311880
10022,5,FF,B,2,SL,B,ball,14.0,0.0,0.0,FF,S,called_strike,9.0,0.0,FT,hit_into_play,8.0,0.0,54.652983,54.652983,54.652983,54.652983,32.440424,32.440424,32.440424,32.440424,12.906592,12.906592,12.906592,12.906592,43.31188,43.31188,43.311880,43.311880
10023,6,FT,B,2,FF,B,ball,14.0,0.0,0.0,SL,B,ball,14.0,0.0,FF,called_strike,9.0,0.0,80.000000,80.000000,80.000000,80.000000,20.000000,20.000000,20.000000,20.000000,0.000000,0.000000,0.000000,0.000000,20.00000,20.00000,20.000000,20.000000
10024,7,FF,S,2,FT,B,ball,14.0,0.0,0.0,FF,B,ball,14.0,0.0,SL,ball,14.0,0.0,80.000000,80.000000,83.333333,83.333333,20.000000,20.000000,16.666667,16.666667,0.000000,0.000000,0.000000,0.000000,20.00000,20.00000,16.666667,16.666667
10025,8,FT,B,2,FF,S,foul,9.0,1.0,0.0,FT,B,ball,14.0,0.0,FF,ball,14.0,0.0,80.000000,80.000000,85.714286,85.714286,20.000000,20.000000,14.285714,14.285714,0.000000,0.000000,0.000000,0.000000,40.00000,40.00000,28.571429,28.571429
10026,9,FT,S,3,FT,B,ball,11.0,0.0,0.0,FF,S,foul,9.0,0.0,FT,ball,14.0,0.0,80.000000,80.000000,87.500000,87.500000,20.000000,20.000000,12.500000,12.500000,0.000000,0.000000,0.000000,0.000000,20.00000,20.00000,25.000000,25.000000
10027,10,CU,S,3,FT,S,called_strike,7.0,0.0,0.0,FT,B,ball,11.0,0.0,FF,foul,9.0,0.0,100.000000,100.000000,88.888889,88.888889,0.000000,0.000000,11.111111,11.111111,0.000000,0.000000,0.000000,0.000000,40.00000,40.00000,33.333333,33.333333


In [438]:
#pitcher_df[cols].tail(20)

## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [439]:
batting_path = '/home/ec2-user/SageMaker/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl'
batting_order_slot_map = pickle.load(open(batting_path, "rb"))

In [440]:
batting_order_slot_map[11] = batting_order_slot_map['pitcher']
batting_order_slot_map[10] = batting_order_slot_map['PH']

In [441]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df

In [442]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: 9
finished w/ slot: 11
finished w/ slot: 10
CPU times: user 473 ms, sys: 4.09 ms, total: 477 ms
Wall time: 472 ms


## Add Pitcher / Batter Matchup History Report

In [443]:
def add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
        
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get all the pitch_types this pitcher has thrown in the past:
        pitch_types = prior_df['pitch_cat'].unique().tolist()
        
        try:
            pitch_types.remove('PO')
        except:
            pass
        
        #get a list of the batters in the current_df
        current_batters = current_df['batter'].unique().tolist()
        
        batters_dict = {}
        
        current_df_list = []
        
        for batter in current_batters:
            batter_df_list = []
            
            #first use subset from prior df
            batter_subset = prior_df[prior_df['batter'] == batter].copy()
            #if pitcher has never faced this batter before:
            if batter_subset.empty:
                #get the left or right handedness of the batter
                stand = current_df[current_df['batter'] == batter]['stand'].values[0]
                #use overall prior tendencies vs left or right handed hitters
                overall, by_count = get_pitch_tendencies(prior_df[prior_df['stand'] == stand])
            else:
                overall, by_count = get_pitch_tendencies(batter_subset)
            batters_dict[batter] = by_count
            
            #now use subset of current_df where batter=batter
            batter_subset = current_df[current_df['batter'] == batter].copy()
            #iterate over the different count_cat types:
            for count_cat in ['ahead', 'behind', 'neutral']:
                count_subset = batter_subset[batter_subset['count_cat'] == count_cat].copy()
                if count_subset.empty:
                    continue
                else:
                    for pitch in pitch_types:
                        try:
                            count_subset['PB_'+pitch] = batters_dict[batter][count_cat][pitch] * 100
                        except KeyError:
                            count_subset['PB_'+pitch] = 0
                            
                current_df_list.append(count_subset)
        
        current_df = pd.concat(current_df_list, sort=False)
        df_list.append(current_df)
                    
    new_df = pd.concat(df_list, sort=False).sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return new_df

In [444]:
%%time
pitcher_df = add_pb_matchup_priors(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 26.3 s, sys: 41.6 ms, total: 26.3 s
Wall time: 26.2 s


In [445]:
pitcher_df17.pitch_type.value_counts()

FT    924
FF    586
SL    450
CH    380
CU    270
Name: pitch_type, dtype: int64

In [446]:
pitcher_df.pitch_type.value_counts()

FT    1927
FF    1215
SL    1199
CH     742
CU     666
Name: pitch_type, dtype: int64

In [447]:
pitcher_df17.pitch_cat.value_counts(normalize=True)

fastball    0.578544
breaking    0.275862
offspeed    0.145594
Name: pitch_cat, dtype: float64

In [448]:
pitcher_df.pitch_cat.value_counts(normalize=True)

fastball    0.546530
breaking    0.324404
offspeed    0.129066
Name: pitch_cat, dtype: float64

In [449]:
pitcher_df.head(1)

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_offspeed,PB_breaking
10018,FF,2018-03-31,15196,92.300003,-1.9879,6.6027,448801.0,502043.0,14.0,NaN,2018.0,-0.5666,1.521,1.1452,2.3646,0.0,0.0,0.0,0.0,1.0,NaN,NaN,488771.0,9.2096,-133.830002,-8.214,-8.9025,30.4104,-12.2472,3.6311,1.5714,NaN,NaN,NaN,92.084,2213.0,6.479,529438.0,54.0219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,L,R,BAL,MIN,B,NaN,Bot,180331_231656,Infield shift,1.0,NaN,False,False,False,True,0,0,0,0,60.324272,27.80822,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.15625,0.265625,0.0,0,0,Kyle Gibson,NaN,ball,NaN,4-Seam Fastball,Standard,00,neutral,fastball,59.780908,64.197531,20.422535,19.341564,19.796557,16.460905,1.0,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,32.440424,32.440424,12.906592,12.906592,43.31188,43.31188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.666667,33.333333,0.0


In [450]:
#pitcher_df[pitcher_df['release_speed'].isna()][['release_speed', 'pitch_type','pitch_name']]

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and July/August 2019 as Test)

In [451]:
# model_features = ['game_date', 'batter', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
#                 'inning','fielder_2', 'sz_top', 'sz_bot', 'game_pk','at_bat_number',
#                 'pitch_number', 'bat_score', 'fld_score', 'stand', 'p_throws',
#                 'inning_topbot', 'if_fielding_alignment','score_diff', 'on_1b_id',
#                 #'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 'L1_ball_right',
#                 'bases_loaded', 'fastball_perc_faced', 'fastball_chase_perc',
#                 'fastball_bip_swung_perc', 'fastball_taken_strike_perc',
#                 'fastball_est_woba', 'fastball_babip', 'fastball_iso_value',
#                 'breaking_perc_faced', 'breaking_chase_perc', 'breaking_bip_swung_perc',
#                 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip',
#                 'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc',
#                 'offspeed_bip_swung_perc', 'offspeed_taken_strike_perc',
#                 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
#                 'pitchout_perc_faced', 'balls','strikes', 'of_fielding_alignment',
#                 '_count', 'count_cat', 'overall_breaking_perc',
#                 'count_cat_breaking_perc', 'overall_fastball_perc',
#                 'count_cat_fastball_perc', 'overall_offspeed_perc',
#                 'count_cat_offspeed_perc', 'pitch_count', 'L1_pitch_type',
#                 'L1_pitch_result', 'L1_pitch_zone', 'L2_pitch_type',
#                 'L2_description', 'L2_pitch_zone','L3_pitch_type',
#                 'L3_description', 'L3_pitch_zone', 'L5_fastball_perc', 
#                 'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
#                 'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc',
#                 'L15_strike_perc', 'batting_order_slot', 'pitcher_AB', 
#                 'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk',
#                 'prev_ab_basehit', 'prev_ab_strikeout', 'PB_fastball', 'PB_breaking',
#                 'PB_offspeed', 'pitch_cat']



In [452]:
drop_cols = ['index','release_speed','release_pos_x','release_pos_z','hit_location',
             'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'hc_x', 'hc_y', 'vx0',
             'vy0', 'vz0', 'ax', 'ay', 'az','hit_distance_sc', 'launch_speed', 'launch_angle',
             'effective_speed', 'release_spin_rate', 'release_extension', 'release_pos_y',
             'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value',
             'woba_denom', 'babip_value', 'iso_value', 'launch_speed_angle', 'home_score',
             'away_score', 'fld_score', 'post_bat_score', 'post_fld_score', 'p_throws',
             'home_team', 'away_team', 'type', 'bb_type', 'sv_id', 'ball_high', 'ball_low',
             'ball_left', 'ball_right', 'batter_swung', 'in_strikezone', 'chased','events',
             'description', 'des', 'pitch_name', 'zone', 'post_away_score', 'post_home_score',
             'batter', 'on_1b_id', 'game_pk']

In [453]:
def extract_date_features(df):
    df = df.copy()
    
    #df['day'] = df['game_date'].dt.day
    df['month'] = df['game_date'].dt.month
    df['year'] = df['game_date'].dt.year
    
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    #df = df.drop('game_date', axis=1)
    return df

#convert datetime into just month and year
model_df = extract_date_features(pitcher_df)

#drop columns before pickling
model_df = model_df.drop(columns=drop_cols)

#rename fielder_2 to catcher_id:
model_df = model_df.rename(columns={'fielder_2':'catcher_id'})

#add bool for whether there are ANY runners on base:
model_df['on_base'] = ((model_df['on_3b'] + model_df['on_2b'] + model_df['on_1b']) > 0) * 1

#fillna for on_1b_id:
#model_df['on_1b_id'] = model_df['on_1b_id'].fillna('N/A')

#fillna for infield and outfield alignment:
model_df[['if_fielding_alignment', 'of_fielding_alignment']] = model_df[['if_fielding_alignment', 'of_fielding_alignment']].fillna('Standard')

#fillna for sz_top and sz_bot
model_df['sz_top'] = model_df['sz_top'].fillna(model_df['sz_top'].median())
model_df['sz_bot'] = model_df['sz_bot'].fillna(model_df['sz_bot'].median())

#fillna for catcher_id w/ most frequent catcher for that pitcher:
catcher = model_df['catcher_id'].value_counts().index[0]
model_df['catcher_id'] = model_df['catcher_id'].fillna(catcher)

#convert cols to categorical:
cat_cols = ['pitch_type', 'balls', 'strikes', 'player_name', 'of_fielding_alignment', '_count', 'count_cat', 'pitch_cat',
              'L1_pitch_type', 'L1_pitch_result', 'L1_description', 'L1_pitch_zone', 'L1_batter_swung', 'L1_chased', 'L2_pitch_type',
              'L2_pitch_result', 'L2_description', 'L2_pitch_zone', 'L2_chased', 'L3_pitch_type', 'L3_description', 'L3_pitch_zone',
              'L3_chased', 'batting_order_slot', 'pitcher', 'outs_when_up', 'inning', 'catcher_id', 'year', 'month',
              'at_bat_number', 'pitch_number', 'pitch_count']
model_df[cat_cols] = model_df[cat_cols].astype('category')


In [454]:
model_df.head()

,pitch_type,game_date,pitcher,on_3b,on_2b,on_1b,outs_when_up,inning,catcher_id,sz_top,sz_bot,at_bat_number,pitch_number,bat_score,stand,inning_topbot,if_fielding_alignment,score_diff,bases_loaded,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_fastball_perc,count_cat_fastball_perc,overall_breaking_perc,count_cat_breaking_perc,overall_offspeed_perc,count_cat_offspeed_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_description,L1_pitch_zone,L1_batter_swung,L1_chased,L2_pitch_type,L2_pitch_result,L2_description,L2_pitch_zone,L2_chased,L3_pitch_type,L3_description,L3_pitch_zone,L3_chased,L5_fastball_perc,L15_fastball_perc,L5_breaking_perc,L15_breaking_perc,L5_offspeed_perc,L15_offspeed_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout,PB_fastball,PB_offspeed,PB_breaking,month,year,on_base
10018,FF,2018-03-31,502043.0,0.0,0.0,0.0,0.0,1.0,488771.0,3.6311,1.5714,1.0,1.0,0.0,L,Bot,Infield shift,1.0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,0,0,Kyle Gibson,Standard,00,neutral,fastball,59.780908,64.197531,20.422535,19.341564,19.796557,16.460905,1.0,N/A,N/A,N/A,-1.0,-1.0,-1.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,32.440424,32.440424,12.906592,12.906592,43.31188,43.31188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.666667,33.333333,0.000000,3,2018,0
10019,FT,2018-03-31,502043.0,0.0,0.0,0.0,0.0,1.0,488771.0,3.5538,1.5042,1.0,2.0,0.0,L,Bot,Infield shift,1.0,0,60.324272,27.808220,28.756958,37.196262,0.519561,0.133333,0.203333,29.661421,34.715027,26.459145,47.881355,0.470353,0.138158,0.111842,10.014306,38.297871,38.679245,24.637682,0.462878,0.156250,0.265625,0.000000,1,0,Kyle Gibson,Standard,10,behind,fastball,59.780908,73.913043,20.422535,6.719368,19.796557,19.367589,2.0,FF,B,ball,14.0,0.0,0.0,N/A,N/A,N/A,-1.0,-1.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,32.440424,32.440424,12.906592,12.906592,43.31188,43.31188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.000000,25.000000,0.000000,3,2018,0
10020,FF,2018-03-31,502043.0,0.0,0.0,0.0,1.0,1.0,488771.0,3.8157,1.8805,2.0,1.0,0.0,R,Bot,Standard,1.0,0,60.031658,30.121817,43.618515,28.175896,0.439553,0.199495,0.176768,30.668777,38.163265,39.130436,28.421053,0.371922,0.163551,0.186916,9.259992,40.397350,40.740742,10.843373,0.425091,0.205479,0.315068,0.039573,0,0,Kyle Gibson,Standard,00,neutral,fastball,56.006006,62.785863,34.459459,30.977131,9.534535,6.237006,3.0,FT,X,hit_into_play,8.0,1.0,0.0,FF,B,ball,14.0,0.0,N/A,N/A,-1.0,-1.0,54.652983,54.652983,32.440424,32.440424,12.906592,12.906592,43.31188,43.31188,2.0,0.0,0.0,0.0,0.0,0.0,0.0,71.428571,0.000000,28.571429,3,2018,0
10021,SL,2018-03-31,502043.0,0.0,0.0,0.0,1.0,1.0,488771.0,3.6814,1.7561,2.0,2.0,0.0,R,Bot,Standard,1.0,0,60.031658,30.121817,43.618515,28.175896,0.439553,0.199495,0.176768,30.668777,38.163265,39.130436,28.421053,0.371922,0.163551,0.186916,9.259992,40.397350,40.740742,10.843373,0.425091,0.205479,0.315068,0.039573,0,1,Kyle Gibson,Standard,01,ahead,breaking,56.006006,46.740859,34.459459,42.130366,9.534535,11.128776,4.0,FF,S,called_strike,9.0,0.0,0.0,FT,X,hit_into_play,8.0,0.0,FF,ball,14.0,0.0,54.652983,54.652983,32.440424,32.440424,12.906592,12.906592,43.31188,43.31188,2.0,0.0,0.0,0.0,0.0,0.0,0.0,33.333333,33.333333,33.333333,3,2018,0
10022,FF,2018-03-31,502043.0,0.0,0.0,0.0,1.0,1.0,488771.0,3.7385,1.8133,2

In [455]:
#model_df.isna().sum()

In [456]:
#model_df.info()

In [457]:
# import category_encoders as ce
# from sklearn.preprocessing import RobustScaler, StandardScaler

# #categoricals cols:


# #one hot encode categorical columns:
# one_hot_cols = ['outs_when_up', 'stand', 'inning_topbot', 'if_fielding_alignment',
#                 'of_fielding_alignment', 'count_cat', 'L1_pitch_type', 
#                 'L1_pitch_result', 'L1_ball_high', 'L1_ball_low', 'L1_ball_left', 
#                 'L1_ball_right','year', 'p_throws', 'L2_pitch_type',
#                 'L2_description', 'L2_pitch_zone','L3_pitch_type',
#                 'L3_description', 'L3_pitch_zone']

# #binary encode categorical columns:
# binary_cols = ['inning', 'batter', 'catcher_id', 'game_pk', 'at_bat_number', 'pitch_number',
#                'bat_score', 'fld_score', 'score_diff', 'on_1b_id', 'balls', 'strikes', 
#                '_count', 'pitch_count', 'L1_pitch_zone', 'batting_order_slot', 
#                'month']
    

# #robust scaler cols:

# numeric_cols = ['fastball_perc_faced', 'fastball_chase_perc', 'fastball_bip_swung_perc', 
#                 'fastball_taken_strike_perc', 'fastball_est_woba', 'fastball_babip', 
#                 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc', 
#                 'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 
#                 'breaking_est_woba', 'breaking_babip', 'breaking_iso_value', 
#                 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc', 
#                 'offspeed_taken_strike_perc','offspeed_est_woba', 'offspeed_babip', 
#                 'offspeed_iso_value','overall_breaking_perc', 'count_cat_breaking_perc', 
#                 'overall_fastball_perc', 'count_cat_fastball_perc', 'overall_offspeed_perc', 
#                 'count_cat_offspeed_perc', 'pitchout_perc_faced', 'L5_fastball_perc', 
#                 'L15_fastball_perc', 'L5_breaking_perc', 'L15_breaking_perc',
#                 'L5_offspeed_perc', 'L15_offspeed_perc', 'L5_strike_perc', 
#                 'L15_strike_perc', 'PB_fastball', 'PB_breaking', 'PB_offspeed', 
#                 'sz_bot', 'sz_top']

# def final_model_preprocess(model_df):
#     df = model_df.copy()
    
#     #one hot encode certain columns
#     one_hot_ce = ce.OneHotEncoder(cols=one_hot_cols, drop_invariant=False, 
#                                   return_df=True, verbose=10, use_cat_names=True)
    
#     one_hot_df = df[one_hot_cols].copy() 
#     one_hot_df = one_hot_ce.fit_transform(one_hot_df)
    
#     #binary encode columns w/ lots of categories
#     binary_ce = ce.BinaryEncoder(cols=binary_cols, return_df=True)
    
#     binary_df = df[binary_cols].copy()
#     binary_df = binary_ce.fit_transform(binary_df)
    
#     #grab the remaining features
#     other_cols = set(df.columns) - set(one_hot_cols + binary_cols)
#     other_df = df[other_cols].copy()
    
#     #merge everything back together
#     df = pd.concat([one_hot_df, binary_df, other_df], axis=1)
#     return df

# model_df = final_model_preprocess(model_df)

In [458]:
model_df.shape

(5749, 91)

In [459]:
#Todo:

# - pickle the model_df and export

# - scale the data (probably w/ robust scaler)

# - split into train and test

# - for non tree based models: encode categorical variables

# - train models and evaluate predictions

In [460]:
vals = list(pitcher_id_dict.keys())
keys = list(pitcher_id_dict.values())
reverse_dict = dict(zip(keys, vals))

pickle_path = '/home/ec2-user/SageMaker/RC-v1.2-Predictive-Modelling/pitcher_df_pickles/'
pitcher_name = reverse_dict[pitcher_id].split()[1]

model_df.to_pickle(path=(pickle_path + pitcher_name + '_df.pkl'),compression='zip')